<a href="https://colab.research.google.com/github/moonjune/test-repo/blob/master/seq2seq.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [81]:
!pip install tensorflow==1.12

In [82]:
from __future__ import absolute_import, division, print_function
# 임포트 된 것을 파악하여 
import tensorflow as tf

tf.enable_eager_execution()

from matplotlib import font_manager, rc

rc('font', family='AppleGothic') #for mac

import matplotlib.pyplot as plt



from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.preprocessing.sequence import pad_sequences

from pprint import pprint
import numpy as np
import os

print(tf.__version__)

1.12.0


In [0]:
sources = [['I', 'feel', 'hungry'],
     ['tensorflow', 'is', 'very', 'difficult'],
     ['tensorflow', 'is', 'a', 'framework', 'for', 'deep', 'learning'],
     ['tensorflow', 'is', 'very', 'fast', 'changing']]
# 4개 케이스, 소스
targets = [['나는', '배가', '고프다'],
           ['텐서플로우는', '매우', '어렵다'],
           ['텐서플로우는', '딥러닝을', '위한', '프레임워크이다'],
           ['텐서플로우는', '매우', '빠르게', '변화한다']]
# 4개 케이스, 타겟

In [84]:
# vocabulary for sources
s_vocab = list(set(sum(sources, [])))

# s_vocab은 왜 정의하는가? 엠베딩 하기 위해 단어별를 원소로 공간을 형성
# sum은 문자열을 합치고 있다. 조인 함수의 경우 어떻게 되는가? 안됨
# 차원이 늘어나면 join으로 합칠 수 없나 봄. 그래서 sum을 사용하게 됨. 뒤의 []는 뭐지...; 변수의 type과 맞춰줘야 하나 봄
# set은 중복 제거, 고유 원소로만 만드나 봄. 얘는 형태가 set 타입으로 변경됨
# 그리고 다시 이걸 list로.. 형태를 복구시켜 줌. unique함수는 numpy나 pandas 쪽이어서 list엔 없음
# 핵심은 문장의 형태인 단어 조합을 해체하여 단어별로 취급하고 중복을 제거하는 것 

s_vocab.sort()
s_vocab = ['<pad>'] + s_vocab
# 거기에 패딩 원소 추가
source2idx = {word : idx for idx, word in enumerate(s_vocab)}
idx2source = {idx : word for idx, word in enumerate(s_vocab)}
# 넘버링 딕셔너리

pprint(source2idx)


{'<pad>': 0,
 'I': 1,
 'a': 2,
 'changing': 3,
 'deep': 4,
 'difficult': 5,
 'fast': 6,
 'feel': 7,
 'for': 8,
 'framework': 9,
 'hungry': 10,
 'is': 11,
 'learning': 12,
 'tensorflow': 13,
 'very': 14}


In [85]:
# vocabularu for targets
t_vocab = list(set(sum(targets,[])))
t_vocab.sort()
t_vocab = ['<pad>', '<bos>','<eos>'] + t_vocab
target2idx = {word : idx for idx, word in enumerate(t_vocab)}
idx2target = {idx : word for idx, word in enumerate(t_vocab)}
#타겟도 마찬가지 처리

pprint(target2idx)

{'<bos>': 1,
 '<eos>': 2,
 '<pad>': 0,
 '고프다': 3,
 '나는': 4,
 '딥러닝을': 5,
 '매우': 6,
 '배가': 7,
 '변화한다': 8,
 '빠르게': 9,
 '어렵다': 10,
 '위한': 11,
 '텐서플로우는': 12,
 '프레임워크이다': 13}


In [0]:
#전처리 함수 정의
def preprocess(sequences, max_len, dic, mode = 'source'):
  assert mode in ['source','target'], 'source와 target 중에 선택해주세요'
  # 뻑내면 나올 오류
  
  if mode == 'source': 
    # 소스 처리를 위한 셀(인코더)
    s_input = list(map(lambda sentence : [dic.get(token) for token in sentence], sequences))
    # 받는 것은 source로 문장 리스트를 받았음. 
    # 그런데 map 함수를 이용하면 리스트 내의 모든 하위 원소에 반응하는 것 같음. [문장1]이 아닌 ['단어1','단어2'] 등
    # dic은 뒤에서 source2idx 인덱스로 들어가는 것
    # 이 과정을 거쳐 s_input은 1차 embedding 완료
    s_len = list(map(lambda sentence : len(sentence), s_input))
    # s_input의 형태는 [['단어1_인덱스','단어2_인덱스'],[단어1인덱스, ..]]이럴건데..
    # 위와 차이는 for 문이 없다는 것이다. 
    # lambda 자체가 인스턴스 함수로 애시당초 시퀀스를 변수로 받는다. 위의 식은 거기에 한번 더 for를 씌워줘서 하윗단까지 가게 한거
    # 여기서는 투입 리스트 원소만 처리하면 되니 for 문을 쓰지 않았고 s_len은 투입받은 리스트의 원소(문장)의 길이를 세 준 것
    s_input = pad_sequences(sequences = s_input, maxlen = max_len, padding = 'post', truncating = 'post')
    # 다시 s_input을 조정한다. pad_sequences는 패딩 씌우는 것
    # max_len에 맞게 10개의 길이로 통일시키기 위해 10개 이하는 뒤에 0(<pad>)을 붙여주고 넘치면 뒤를 잘라버림
    return s_len, s_input
    # 결과로 s_len과 s_input이 출력됨
    
  elif mode == 'target':
    # 타겟 처리를 위한 셀(디코더)
    # 투입물
    t_input = list(map(lambda sentence : ['<bos>'] + sentence + ['<eos>'], sequences))
    # 앞에 <bos>가 붙어서 문장의 시작을 알려서 y1이 문장의 첫 단어가 되도록 해준다. 
    # 뒤에 <eos>로 문장이 종료되었음을 선언
    # 이것은 번역이나 대화를 위한 세팅. 디코더가 만들 문장은 인코더 셀을 다 지난 후 bos로 시작하고 eos로 종료시킴
    t_input = list(map(lambda sentence : [dic.get(token) for token in sentence], t_input))
    # s_input과 비슷하다. <bos>,<eos>가 추가된다.    
    t_len = list(map(lambda sentence : len(sentence), t_input))
    t_input = pad_sequences(sequences = t_input, maxlen = max_len, padding = 'post', truncating = 'post')
    # <bos>,<eos>가 추가되었지만 maxlen은 동일함. 이건 나중에 살펴보자
    # target은 번역되는 대상
    
    # output
    t_output = list(map(lambda sentence : sentence + ['<eos>'], sequences))
    # 타겟 아웃풋은 먼저 문장단위에 ['eos]만 붙여 투입한다.
    t_output = list(map(lambda sentence : [dic.get(token) for token in sentence], t_output))
    # target2idx를 쓰지만 역시 숫자로 이뤄진 결과를 도출한다.
    t_output = pad_sequences(sequences = t_output, maxlen = max_len, padding = 'post', truncating = 'post')
    
    return t_len, t_input, t_output
  
  #전처리는 엠베딩 과정, 그 과정에서 <pad>등을 포함한 엠베딩을 만들고 입력 문자를 형식에 맞춰 뱉어내도록 하는 것
    

In [87]:
# 소스 전처리(위의 정의 함수)
s_max_len = 10
s_len, s_input =preprocess(sequences = sources,
                           max_len = s_max_len, dic = source2idx, mode = 'source')
print(s_len, s_input)

[3, 4, 7, 5] [[ 1  7 10  0  0  0  0  0  0  0]
 [13 11 14  5  0  0  0  0  0  0]
 [13 11  2  9  8  4 12  0  0  0]
 [13 11 14  6  3  0  0  0  0  0]]


In [88]:
# 타겟 전처림
t_max_len = 12
#타겟의 길이는 12. 아마도 패딩들을 고려한 것으로 보임
t_len, t_input, t_output = preprocess(sequences = targets,
                                      max_len = t_max_len, dic = target2idx, mode = 'target')
print(t_len, t_input, t_output)


[5, 5, 6, 6] [[ 1  4  7  3  2  0  0  0  0  0  0  0]
 [ 1 12  6 10  2  0  0  0  0  0  0  0]
 [ 1 12  5 11 13  2  0  0  0  0  0  0]
 [ 1 12  6  9  8  2  0  0  0  0  0  0]] [[ 4  7  3  2  0  0  0  0  0  0  0  0]
 [12  6 10  2  0  0  0  0  0  0  0  0]
 [12  5 11 13  2  0  0  0  0  0  0  0]
 [12  6  9  8  2  0  0  0  0  0  0  0]]


In [0]:
# 하이퍼 파라미터
epochs = 100 # 몇번 돌릴 건가
batch_size = 4 # 한번에 장전할 문장 수(전체가 4개니..)
learning_rate = .005 # 학습률이 학습률이지 뭐..
total_step = epochs / batch_size 
# 이게 뭐지 대체;;; 의미적으로는 전체 배치가 도는 스텝인거고 epoch는 한 문장의 돌리는 의미인가?
buffer_size = 100 # 얘는 아직 모르겠음 ㅎㅎ
n_batch = buffer_size //batch_size # 배치의 개수는 버퍼 크기 나누기 배치 사이즈..
embedding_dim = 32 # 얘는 왜 고정된 
units = 32 # hidden size인듯

#input
data = tf.data.Dataset.from_tensor_slices((s_len, s_input, t_len, t_input, t_output))
#데이터를 메모리에 넣어주는 기능. 0번째 차원의 크기가 동일해야 함(케이스의 수 정도)
data = data.shuffle(buffer_size = buffer_size)
# buffer 크기 기준으로 섞기 시작, 
data = data.batch(batch_size = batch_size)
#s_mb_len, s_mb_input, t_mb_len, t_mb_input, t_mb_output = iterator.get_next()
# batch size 대로 데이터 준비

# 머신러닝 타입 model 에 데이터를 넣기 위해 직관적인거 같다. 텐서들 한번에 넣어주고, 각 텐서들을 동일한 기준으로 섞고, 배치 준비

In [0]:
def gru(units):
  # GPU가 있다면 CuDNNGRU 사용 추천(3배 빠름)
  # 이 코드는 자동적으로 사용해줍니다.
  # GRU 는 gate recurrent unit의 약자. GPU를 사용한다면 여기서 사용하는 것으로 보인다. 
  if tf.test.is_gpu_available():
    return tf.keras.layers.CuDNNGRU(units,
                                    return_sequences = True,
                                    return_state = True,
                                    recurrent_initializer = 'glorot_uniform')
  else:
    return tf.keras.layers.GRU(units, # 양의 정수로 아웃풋 공간의 차원을 의미
                               return_sequences = True, # 이항변수이며, 마지막 sequence만 할지, 아니면 전체 시퀀스를 반환할지 
                               return_state = True, # 결과물에 최종 스테이트를 반환할지
                               recurrent_activation = 'sigmoid', # rnn의 활성화 함수. 여기선 시그모이드 함수로
                               recurrent_initializer = 'glorot_uniform') # 가중치 행렬을 초기화하는 방법, 
  
  # rnn의 게이트를 담당.. 상세한건 나중에

In [0]:
class Encoder(tf.keras.Model): # 인코딩(정보를 받고 투입된 정보에 대한 어떤 context를 생성하는 클래스)
  def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
    super(Encoder, self).__init__()  
    # super는 기본적으로 부모의 함수를 가져오는 것이라고 한다.
    # Encoder가 부모로 삼고 있는 클래스는 tf.keras.Model이다. 
    # super(Encoder,self)는 그럼 뭘 의미하나... 일단은 넘어갈까;; 
    self.batch_sz = batch_sz # 1회당 넣을 배치 크기
    self.enc_units = enc_units # 인코더 유닛 수(32)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim) # 엠베딩 레이어에 들어갈 단어 열 크기(모름),
    # 그런데 말입니다. embedding이란 무엇일까요?
    # 일단 keras.layers.embedding 이란 레이어층을 말한다
    self.gru = gru(self.enc_units) #gru는 앞서 정의한 gru이고 인코더 유닛 32개 배치
    
  def call(self, x, hidden):
    x = self.embedding(x) # 인코더 셀은 앞의 변수와 x, hidden을 받는다. x는 엠베딩 레이어에 x가 들어간 것? 
    # 이 의미는 embedding layer에 x를 통과시키는 것이라고 한다. 
    # 분명 각 rnn은 embedding 층, 신경망층이 합쳐져 있으므로 납득은 감
    output, state = self.gru(x, initial_state = hidden) # 아웃풋과 state는 gru에서 받는 최종 컨텍스트 값과 state 값을 받음
    
    return output, state
    # 그걸 아웃풋으로 내놓음
    # call은 다른 의미를 이미 가진 함수로 보임. 
    # 미리 소환된 class Encoder는 초기화 값을 가지며 소환된다. 다른 명령 없이 바로 (x)를 넣으면 발동.
 
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.enc_units)) 
  # 히든 스테이트를 초기화, 히든 스테이트는 배치 사이즈를 행으로, 인코더 유닛수를 열로 하는 0행렬
  # 여기서 의미하는 배치 사이즈는 뭘까.. 
  
  # 인코더는 단어 사이즈(인덱스화 한 사전의 길이), 엠베딩 차원(32 지정), units(32 지정), batch_size(4개 지정)을 초기값으로 받음
  # 받은 초기값을 기반으로 gru셀과 히든 스테이트를 준비한다.
  # 그리고 만들어진 인코더 클래스+(x)를 넣으면 할동 후 output과 state를 출력해 줌

In [0]:
class Decoder(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
    super(Decoder, self).__init__()
    self.batch_sz = batch_sz
    self.dec_units = dec_units
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = gru(self.dec_units)
    self.fc = tf.keras.layers.Dense(vocab_size)
    # 디코더 역시 비슷비슷
    # 그러나 fully connected 초기화가 있음
    # 
    
  def call(self, x, hidden, enc_output):
    
    x = self.embedding(x)
    output, state = self.gru(x, initial_state = hidden)
    
    # output shape는 배치 사이즈에 *1, 히든 사이즈(32)...?
    output = tf.reshape(output, (-1, output.shape[2]))
    
    x = self.fc(output)
    
    return x, state
  
  def initialize_hidden_state(self):
    return tf.zeros((self.batch_sz, self.dec_units))
    

In [0]:
encoder = Encoder(len(source2idx), embedding_dim, units, batch_size)
decoder = Decoder(len(target2idx), embedding_dim, units, batch_size)

def loss_function(real, pred):
  mask = 1 - np.equal(real, 0)
  loss_ = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = real, logits = pred) * mask
  
  #print("real: {}".format(real))
  #print("pred: {}".format(pred))
  #print("mask: {}".format(mast))
  #print("loss: {}".format(tf.reduce_mean(loss)))
  
  return tf.reduce_mean(loss_)

# creating optimizer
optimizer = tf.train.AdamOptimizer()

# creating chech point (Object-based saving)
checkpoint_dir = '.data_out/traing_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt')
checkpoint = tf.train.Checkpoint(optimizer = optimizer,
                                 encoder = encoder,
                                 decoder = decoder)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir = checkpoint_dir)

In [111]:
EPOCHS = 100

for epoch in range(EPOCHS):
  
  hidden = encoder.initialize_hidden_state()
  total_loss = 0
  
  for i , (s_len, s_input, t_len, t_input, t_output) in enumerate(data):
    loss = 0
    with tf.GradientTape() as tape:
      enc_output, enc_hidden = encoder(s_input, hidden)
      
      dec_hidden = hidden
      
      dec_input = tf.expand_dims([target2idx['<bos>']] * batch_size, 1)
      
      # Teacher Forcing: feeding the ratget as the next input
      for t in range(1, t_input.shape[1]):
        
        predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_output)
        
        loss += loss_function(t_input[:, t], predictions)
        
        dec_input = tf.expand_dims(t_input[:, t], 1) # using teacher forcing?
        
    batch_loss = (loss / int(t_input.shape[1]))
    
    total_loss += batch_loss
    
    variables = encoder.variables + decoder.variables
    
    gradient = tape.gradient(loss, variables)
    
    optimizer.apply_gradients(zip(gradient, variables))
    
  if epoch % 10 == 0:
    #save model every 10 epoch
    print('Epoch{} || Loss {:.4f} || Batch Loss {:.4f}'.format(epoch,
                                                               total_loss / n_batch,
                                                              batch_loss.numpy()))
    checkpoint.save(file_prefix = checkpoint_prefix)
   

Epoch0 || Loss 0.0229 || Batch Loss 0.5722
Epoch10 || Loss 0.0196 || Batch Loss 0.4899
Epoch20 || Loss 0.0161 || Batch Loss 0.4033
Epoch30 || Loss 0.0130 || Batch Loss 0.3250
Epoch40 || Loss 0.0106 || Batch Loss 0.2646
Epoch50 || Loss 0.0088 || Batch Loss 0.2204
Epoch60 || Loss 0.0076 || Batch Loss 0.1898
Epoch70 || Loss 0.0068 || Batch Loss 0.1696
Epoch80 || Loss 0.0062 || Batch Loss 0.1562
Epoch90 || Loss 0.0059 || Batch Loss 0.1472


In [112]:
# resore checkpoint

checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [0]:
sentence = 'I feel hungry'

In [114]:
def prediction(sentence, encoder, decoder, inp_lang, targ_lang, max_length_inp, max_length_targ):
  
  inputs = [inp_lang[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs], maxlen = max_length_inp, padding = 'post')
  inputs = tf.convert_to_tensor(inputs)
  
  result = ''
  
  hidden = [tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)
  
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang['<bos>']], 0)
  
  for t in range(max_length_targ):
    predictions, dec_hidden = decoder(dec_input, dec_hidden, enc_out)
    
    predicted_id = tf.argmax(predictions[0]).numpy()
    
    result += idx2target[predicted_id] + ' '
    
    if idx2target.get(predicted_id) == '<eos>':
      return result, sentence
    
    #the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)
    
  return result, sentence

result, output_sentence = prediction(sentence, encoder, decoder, source2idx, target2idx, s_max_len, t_max_len)

print(sentence)
print(result)
      

I feel hungry
텐서플로우는 매우 빠르게 변화한다 <eos> 
